In [6]:
from prettytable import PrettyTable
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [93]:
from datetime import date, timedelta
import json
import requests


class DefaultFilter:
    def filter(self, data):
        return data


class ReFilter(DefaultFilter):
    def __init__(self, pattern, key):
        self.key = key
        self.pattern = pattern
        self.re_pattern = re.compile(pattern)

    def filter(self, data):
        return [row for row in data if self.re_pattern.search(row[key])]


class DefaultReader:
    def __init__(self):
        self.format = "xml"

    def decode(self):
        return request.content


class JsonReader:
    def __init__(self):
        self.format = "json"

    def decode(self, request):
        return json.loads(request.content)


class Matomo:
    def __init__(self, reader=DefaultReader, period=None, filter=None, **init_params):
        self.url = "https://stats.data.gouv.fr/index.php"
        self.parameters = {
            "module": "API",
            "method": "Actions.getPageUrls",
            "idSite": 236,
            "period": "day",
            "date": "today",
            "token_auth": "e13428c4dc3dcb05b0b5cb0cbbf31714",
            "expanded": 1,
            "flat": 1,
        }
        self.set_period(*period)
        self.parameters.update(init_params)
        self.reader = reader()
        self.parameters.update({"format": self.reader.format})
        if filter:
            self.filter = filter
        else:
            self.filter = lambda x: x

    def set_period(self, *args):
        if args is None:
            return
        start, end = min(args), max(args)
        self.parameters.update({
            "period": "range",
            "date": f"{start.strftime('%Y-%m-%d')},{end.strftime('%Y-%m-%d')}"
        })

    def send(self):
        request = requests.get(self.url, params=self.parameters)
        return self.reader.decode(request)

In [70]:
mato = Matomo(reader=JsonReader, period=(date(2022, 6, 20), date(2022, 6, 1)))
data = mato.send()
data

[{'label': '/project/diagnostic/etape/ - Autres',
  'nb_visits': 354,
  'nb_hits': 447,
  'sum_time_spent': 3137,
  'nb_hits_with_time_network': 447,
  'min_time_network': 1.5370000000000001,
  'max_time_network': 1.8679999999999999,
  'nb_hits_with_time_server': 447,
  'min_time_server': 82.03400000000002,
  'max_time_server': 87.12500000000003,
  'nb_hits_with_time_transfer': 440,
  'min_time_transfer': 0.48700000000000004,
  'max_time_transfer': 0.927,
  'nb_hits_with_time_dom_processing': 291,
  'min_time_dom_processing': 2.066,
  'max_time_dom_processing': 5.798,
  'nb_hits_with_time_dom_completion': 1,
  'min_time_dom_completion': 0,
  'max_time_dom_completion': 0,
  'nb_hits_with_time_on_load': 0,
  'min_time_on_load': None,
  'max_time_on_load': None,
  'sum_daily_nb_uniq_visitors': 403,
  'exit_nb_visits': 4,
  'sum_daily_exit_nb_uniq_visitors': 4,
  'nb_hits_following_search': 9,
  'entry_nb_visits': 1,
  'entry_nb_actions': 5,
  'entry_sum_visit_length': 67,
  'entry_bounce_

In [71]:
x = PrettyTable(field_names=["page", "visits", "visiteurs"], align="l")
x.add_rows((row["label"][:75], row["nb_hits"], row["nb_visits"]) for row in data)
print(x)

+-----------------------------------------------------------------------------+--------+-----------+
| page                                                                        | visits | visiteurs |
+-----------------------------------------------------------------------------+--------+-----------+
| /project/diagnostic/etape/ - Autres                                         | 447    | 354       |
| /                                                                           | 418    | 298       |
| /project/ - Autres                                                          | 875    | 154       |
| /project/diagnostic/etape/1                                                 | 195    | 134       |
| /project/diagnostic/dates                                                   | 217    | 128       |
| /users/signin/?next=/project/                                               | 26     | 25        |
| /project/                                                                   | 28     | 19

Plan de taggage

page => indicateur
index => visites de l'app

In [97]:
import re
from collections import defaultdict

re_map = re.compile(r"(consommation|couverture|synthesis|usage|artificialisation)")

total = defaultdict(lambda: 0)
for row in data:
    match = re_map.search(row["label"])
    if match:
        total[match.group(0)] += row["nb_hits"]
        

x = PrettyTable(field_names=["Rapport", "Hits"], align="l")
x.add_rows((k, v) for k, v in total.items())
print(x)

+-------------------+------+
| Rapport           | Hits |
+-------------------+------+
| consommation      | 47   |
| synthesis         | 21   |
| couverture        | 3    |
| artificialisation | 4    |
| usage             | 1    |
+-------------------+------+


In [66]:
[row["label"] for row in data]

['/',
 '/project/diagnostic/dates',
 '/project/diagnostic/etape/1',
 '/faq/foire-aux-questions-pedagogique/',
 '/project/2483/',
 '/project/2483/tableau-de-bord/artificialisation',
 '/project/2483/tableau-de-bord/consommation',
 '/project/2483/tableau-de-bord/consommation?relative=true',
 '/project/2483/tableau-de-bord/couverture',
 '/project/2483/tableau-de-bord/synthesis',
 '/project/2483/tableau-de-bord/telechargement',
 '/project/2483/tableau-de-bord/usage',
 '/project/2485/',
 '/project/2485/map',
 '/project/2485/tableau-de-bord/consommation',
 '/project/2485/tableau-de-bord/synthesis',
 '/project/diagnostic/etape/1?csrfmiddlewaretoken=7jIUnKU54jltsRjOF4nPxfhII9SKw6iZvu2yp0BGpiJlgUjK8PazoaMc871Qy4uM&region=6',
 '/project/diagnostic/etape/1?csrfmiddlewaretoken=FgUeaBoLLnXKaM54Ida2YGTL2anENieqaUMyiVx2TLLfcYEH7DxaI1yfKXzJ3Nho&region=13',
 '/project/diagnostic/etape/1?csrfmiddlewaretoken=Q852qrzdoimGwNJqVCiuJsRoo9nl8DMalMXmyLIuwGabyZi3k2FCtNwS6Wzqo8P8&epci=267&departement=13',
 '/proj

In [78]:
re_map.search("/project/2396/map").group(0)

'/map'

In [87]:
today = date.today()
today

datetime.date(2022, 6, 21)

In [99]:
first = date(year=today.year - 1, month=today.month + 1, day=1)
first

datetime.date(2021, 7, 1)

In [3]:
from django.db.models import Subquery, OuterRef, F
from project.models import Request, Emprise
from public_data.models import Region
from utils.colors import get_color_gradient

project_list = Request.objects.all().values_list("project", flat=True)
emprise_list = 
qs_regions = Region.objects.all().annotate(total=Subquery(emprise_list[:1])).order_by("total")
colors = get_color_gradient(scale=qs.regions.count())
for region in qs_regions:
    region.map_color = colors.pop(0)
for region in qs_regions:
    print(region.name, region.map_color)

ValueError: This queryset contains a reference to an outer query and may only be used in a subquery.

In [18]:
project_list = Request.objects.all().values_list("project", flat=True)
sub_qs = Emprise.objects.filter(mpoly__contained=OuterRef("mpoly")).annotate(total=Count("project_id", distinct=True)).values("total")[:1]
qs = (
    Region.objects
    .annotate(total=Subquery(sub_qs))
    .order_by("total")
)

for region in qs:
    print(region.name, region.total)

Occitanie 1
Nouvelle-Aquitaine 1
Bretagne None
Centre-Val de Loire None
Corse None
Grand Est None
Hauts-de-France None
Normandie None
Pays de la Loire None
Provence-Alpes-Côte d'Azur None
Auvergne-Rhône-Alpes None
Île-de-France None
Bourgogne-Franche-Comté None


Occitanie 1
Nouvelle-Aquitaine 1
Provence-Alpes-Côte d'Azur 1
Auvergne-Rhône-Alpes 1
Corse None
Grand Est None
Hauts-de-France None
Normandie None
Pays de la Loire None
Île-de-France None
Bourgogne-Franche-Comté None
Bretagne None
Centre-Val de Loire None


In [13]:
Emprise.objects.filter(
    # project__in=project_list,
    mpoly__intersects=region.mpoly
).annotate(total=Count("project_id", distinct=True)).values("total")[:1]

<QuerySet [{'total': 1}]>

In [12]:
Request.objects.all().values_list("project", flat=True)

<QuerySet []>